In [23]:
# Import the numpy library and assign it an alias "np"
import numpy as np

# Import various classes and functions from the Qiskit framework
from qiskit import QuantumCircuit, transpile, Aer, execute

# Enable inline plotting in Jupyter notebooks
%matplotlib inline

# Import the QasmSimulator from the Aer provider
from qiskit.providers.aer import QasmSimulator

# Import the job_monitor function from the qiskit.tools.monitor module
from qiskit.tools.monitor import job_monitor

# Load your IBM Quantum account using your API token saved in "token.txt"
IBMQ.save_account(open("token.txt", "r").read())


configrc.store_credentials:WARNING:2023-02-26 01:18:44,566: Credentials already present. Set overwrite=True to overwrite.


In [24]:
# Import necessary modules from Qiskit
from qiskit import IBMQ

# Load IBM Quantum Experience account credentials
# This function reads your API token from a configuration file and sets up an interface
# between your computer and IBM's quantum computing resources
IBMQ.load_account()


ibmqfactory.load_account:WARNING:2023-02-26 01:18:45,584: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [25]:
# Import necessary modules from Qiskit
from qiskit import IBMQ

# Connect to the IBM Quantum Experience provider and get a list of available backends
provider = IBMQ.get_provider("ibm-q")
for backend in provider.backends():

    # Check how many qubits are available for the current backend
    try:
        qubit_count = len(backend.properties().qubits)
    except:
        # If the backend is simulated, set the qubit count to "simulated"
        qubit_count = "simulated"

    # Print the name of the backend, the number of qubits available, and the number of pending jobs
    print(f"{backend.name()} has {backend.status().pending_jobs} queued and {qubit_count} qubits")


ibmq_qasm_simulator has 1 queued and simulated qubits
ibmq_lima has 0 queued and 5 qubits
ibmq_belem has 41 queued and 5 qubits
ibmq_quito has 49 queued and 5 qubits
simulator_statevector has 1 queued and simulated qubits
simulator_mps has 1 queued and simulated qubits
simulator_extended_stabilizer has 1 queued and simulated qubits
simulator_stabilizer has 1 queued and simulated qubits
ibmq_jakarta has 235 queued and 7 qubits
ibmq_manila has 84 queued and 5 qubits
ibm_lagos has 133 queued and 7 qubits
ibm_nairobi has 68 queued and 7 qubits
ibm_perth has 151 queued and 7 qubits
ibm_oslo has 83 queued and 7 qubits


In [26]:
def find_largest_number(number1, number2):
    # Set up a quantum circuit with 2 integer qubits and 1 control qubit
    qc = QuantumCircuit(3, 2)

    # Encode the two integers as binary numbers on separate sets of qubits
    qc.x(0) if number1 == 1 else None
    qc.x(1) if number2 == 1 else None
    qc.barrier()

    # Apply a Hadamard gate to the control qubit to put it into a superposition of |0⟩ and |1⟩
    qc.h(2)

    # Apply a controlled-Z gate to encode the integers in the phase of the state
    qc.cz(2, 0)

    # Apply the inverse Quantum Fourier Transform (QFT) to the b qubits
    qc.barrier()
    qc.h(1)
    qc.cp(np.pi/2, 1, 0)
    qc.h(0)

    # Measure the b qubits and apply classical post-processing to obtain the phase
    qc.measure([0,1], [0,1])
     # Get the backend (quantum computer) to execute the circuit on
    backend = provider.get_backend("ibmq_lima")

    # Execute the quantum circuit on the backend, taking a large number of shots
    # to obtain accurate measurement results
    job = execute(qc, backend=backend, shots=2048)

    # Monitor the status of the job on the backend
    job_monitor(job)

    # Get the result of the job, which includes the measurement counts for each qubit
    result = job.result()

    # Get the measurement results
    counts = result.get_counts(qc)
    p01 = counts.get('01', 0) / 2048  # probability of measuring 01
    p10 = counts.get('10', 0) / 2048  # probability of measuring 10

    # Determine which integer is larger based on the measurement results
    if p01 > p10:
        return number2
    else:
        return number1


In [28]:
a = 5
b = -6
largest = find_largest_number(a,b)
print(f"The largest number from {a} and {b} is {largest}")


Job Status: job has successfully run
The largest number from 5 and -6 is 5
